## MCP

In [ ]:
import boto3, os
from dotenv import load_dotenv
cognito_client = boto3.client('cognito-idp')

assert load_dotenv(dotenv_path="../.env_cognito")
client_id = os.getenv("app_client_id")

# Authenticate User and get Access Token
auth_response = cognito_client.initiate_auth(
    ClientId=client_id,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': 'testuser',
        'PASSWORD': 'MyPassword123!'
    }
)
bearer_token = auth_response['AuthenticationResult']['AccessToken']
bearer_token

: 

In [2]:
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")

    load_dotenv(dotenv_path=".env_mcp")
    agent_arn = os.getenv("AgentARN")
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

await main()

Using AWS region: us-east-1

Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A986603626835%3Aruntime%2Fmcp_server_s3_upload-q1TTam6gmI/invocations?qualifier=DEFAULT
Headers configured ✓

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 upload_file
   Description: Upload a file to S3 bucket and return a presigned URL with expiration
    
    Args:
        file_name: Name of the file to upload (including extension)
        file_content: Content of the file as a string
    
    Returns:
        presigned S3 URL of the uploaded file
    
   Parameters: ['file_name', 'file_content']

✅ Successfully connected to MCP server!
Found 1 tools available.


## Gateway

In [3]:
load_dotenv(dotenv_path=".env_cognito")
load_dotenv(dotenv_path=".env_gateway")

user_pool_id = os.getenv("pool_id")
m2m_client_id = os.getenv("m2m_client_id")
m2m_client_secret = os.getenv("m2m_client_secret")
scope_string = os.getenv("scope_string")
gatewayURL = os.getenv("gateway_url")
boto_session = Session()
REGION = boto_session.region_name

In [4]:
import utils

print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
token_response = utils.get_token(user_pool_id, m2m_client_id, m2m_client_secret, scope_string, REGION)
token = token_response["access_token"]
print("Token response:", token)

from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL, headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

from strands import Agent
import logging

# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all tools available to you")
    agent("What is the weather in mars")

Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes
3atbr3degon07fp0f80ruas817
uc4d05b1959kfu4q7si6c3d9d9dp645v7323m4b7rmn0ap2ulae
Token response: eyJraWQiOiJ3RTh2dm5PbWR6TEMyVmVLM0toemJ0cDhnTnpwTHJzckZBZmdyMVhtMW1BPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIzYXRicjNkZWdvbjA3ZnAwZjgwcnVhczgxNyIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoic3RyYW5kcy1kZW1vLXJlc291cmNlLXNlcnZlci1pZFwvZ2F0ZXdheTpyZWFkIHN0cmFuZHMtZGVtby1yZXNvdXJjZS1zZXJ2ZXItaWRcL2dhdGV3YXk6d3JpdGUiLCJhdXRoX3RpbWUiOjE3NTMzNDY4MDQsImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC51cy1lYXN0LTEuYW1hem9uYXdzLmNvbVwvdXMtZWFzdC0xX2JGZzRZalBkUiIsImV4cCI6MTc1MzM1MDQwNCwiaWF0IjoxNzUzMzQ2ODA0LCJ2ZXJzaW9uIjoyLCJqdGkiOiIxM2UyZDdlNi02ZDdmLTQxMmMtODQ3ZS0wOWNjNGE0M2IzYjYiLCJjbGllbnRfaWQiOiIzYXRicjNkZWdvbjA3ZnAwZjgwcnVhczgxNyJ9.L_pnOmrfJFOU1qJ2T9hLD9N2tWk3-WkDktuOzX86sDobFAew1uAi-plw3SDpfjEVKa7F-omal-zEqu38NKJRtmKaa2inhSd6f48uAbNYI0czLSM7WZR_Qu2nAEZwrKrgWtHIYiVD4MxBkRRAEYyIJqrd4blk1DdJEFa23

INFO | strands.telemetry.metrics | Creating Strands MetricsClient


Tools loaded in the agent are ['DemoOpenAPITargetS3NasaMars___getInsightWeather']
<thinking> The user has asked for a list of all available tools. I should provide this information directly as it is a simple request that does not require the use of any tools. </thinking>

Sure, here is the list of all tools available to me:

1. **DemoOpenAPITargetS3NasaMars___getInsightWeather**
   - **Description:** Retrieve latest InSight Mars weather data.
   - **Parameters:**
     - `ver` (string): API version string. (only "1.0" supported)
     - `feedtype` (string): Response format (only "json" is supported).
   - **Required Parameters:** `feedtype`, `ver`<thinking> The user has asked for the current weather on Mars. I should use the `DemoOpenAPITargetS3NasaMars___getInsightWeather` tool to retrieve the latest weather data from Mars. </thinking>


Tool #1: DemoOpenAPITargetS3NasaMars___getInsightWeather
Here is the current weather on Mars:

- **Season:** Fall
- **Northern Season:** Mid winter
- *